In [ ]:
import gc
import sys
import cv2
import progressbar as pb
import matplotlib.pyplot as plt
import numpy as np
from io import BytesIO
from common import *

In [ ]:
# Set File Name
fileName = "csi_test.csv"
avgBeginTime = 44.416500
avgEndTime = 46.624219

In [ ]:
# Load CSV & Calculate necessary frames
print("[1/6] Loading CSV ( " + fileName + " ) ...")
nx = getNx(fileName)
print("[2/6] Calculating necessary frames...")
frameIdx = getNecessaryFrameIdx(nx)
frameLen = len(frameIdx)

In [ ]:
# Calculate avarage CSI and differences
print("[3/6] Calculating average CSI...")
cutNx = None
beginIdx = 0
for idx, row in enumerate(nx):
    if row[0] >= avgBeginTime:
        beginIdx = idx
        break
for idx, row in reversed(list(enumerate(nx))):
    if row[0] < avgEndTime:
        cutNx = nx[beginIdx:idx, :]
        break
avgAmp = np.mean(cutNx, axis=0)
print("[4/6] Calculating differences...")
diffNx = np.copy(nx) - avgAmp

In [ ]:
# Calculate accumulated
print("[5/6] Calculating accum...")
acNx = {}
tsFPS = getFPS()
xm, ym = np.meshgrid(np.array(range(30)), np.linspace(-10.0, 10.0, 600))
for i in range(0, int((nx.shape[1] - 1) / 60)):
    tsFrame = 2 * getFPS()
    acNx[i] = {}
    print("  -- Initialize", i, "TRP...")
    with pb.ProgressBar(max_value=len(frameIdx)) as bar:
        for j in range(len(frameIdx)):
            acNx[i][j] = np.zeros((600, 30))
            bar.update(j)
    print("  -- Calculate", i, "TRP...")
    with pb.ProgressBar(max_value=len(frameIdx)) as bar:
        bar.update(0)
        for idx, fidx in enumerate(frameIdx):
            if idx + tsFrame >= len(frameIdx):
                tsFrame = len(frameIdx) - idx - 1
            for j in range(tsFrame):
                for k in range(30):
                    tenx = np.zeros(600)
                    nmx = int(np.ceil(diffNx[fidx, (1 + 30 * i):(31 + 30 * i)][k] * 30 + 300))
                    bold = (abs(diffNx[fidx, (1 + 30 * i):(31 + 30 * i)][k]) * (tsFrame - j) / tsFrame)
                    if nmx >= 600:
                        tenx[300:599] = bold
                    elif nmx >= 300:
                        tenx[300:nmx] = bold
                    elif nmx < 0:
                        tenx[0:300] = bold
                    else:
                        tenx[nmx:300] = bold
                    acNx[i][idx + j][:, k] += tenx
            bar.update(idx)

In [ ]:
# Calculate graphs
# HEY, BELIEVE YOUR VIRTUAL MEMORY!
print("[6/6] Creating graphs in cv format and save them...")
for i in range(0, int((nx.shape[1] - 1) / 60)):
    frames = []
    barIdx = 0
    gc.collect()
    print("  -- Processing " + str(i + 1) + " th Tx-Rx Pair...")
    with pb.ProgressBar(max_value=frameLen) as bar:
        for l, p in enumerate(frameIdx):
            pltFigure = plt.figure()
            pltImage = BytesIO()
            plt.colorbar(plt.contourf(xm, ym, acNx[i][l], figure=pltFigure, vmin=0, vmax=250, cmap='jet'))
            plt.xlabel('Subcarriers Group', figure=pltFigure)
            plt.ylabel('Amplitude Diff Accum (dBm)', figure=pltFigure)
            pltFigure.savefig(pltImage, format='png')
            plt.close(pltFigure)
            del pltFigure
            pltImage.seek(0)
            cvImage = cv2.imdecode(np.asarray(
                bytearray(pltImage.read()), dtype=np.uint8), cv2.IMREAD_COLOR)
            frames.append(cvImage)
            barIdx += 1
            bar.update(barIdx)
    # Save as mp4 video
    videoName = conf.VID_NAME.format(
        fileName[:fileName.index('.')], "amp-accum" + str(i + 1))
    print("  -- Saving in " + videoName + " ...")
    saveVideo(videoName, frames)
    print("  -- Delete old data...")
    del frames
print("Finished!")

In [ ]:
#Finished